## Ner talk

In [1]:
from IPython.display import Image
from IPython.core.display import display, HTML
import pandas as pd
from collections import Counter
import random
from spacy import displacy
import json

from spacy.training import offsets_to_biluo_tags
import spacy
import eli5
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import f1_score

In [2]:
with open('restauranttrain.bio', 'r') as f:
    for line in f.readlines()[:30]:
        print(line)

B-Rating	2

I-Rating	start

O	restaurants

O	with

B-Amenity	inside

I-Amenity	dining



O	34



B-Rating	5

I-Rating	star

O	resturants

B-Location	in

I-Location	my

I-Location	town



O	98

B-Restaurant_Name	hong

I-Restaurant_Name	kong

O	restaurant

B-Price	reasonable

O	prices



O	a

O	great

O	lunch

O	spot

O	but

B-Hours	open

I-Hours	till



In [3]:
# switch column places
with open('restauranttrain.bio', 'r') as f:
    with open('restauranttrain_updated.bio', 'w') as w:
        for line in f.readlines():
            if line == '\n':
                w.write(line)
            else:
                w.write('\t'.join(line.strip().split('\t')[::-1]) + '\n')

In [4]:
# split into train and validation
count = 0
with open('restauranttrain.bio', 'r') as f:
    with open('restauranttrain_updated_train.bio', 'w') as w1:
        with open('restauranttrain_updated_valid.bio', 'w') as w2:
            for line in f.readlines():
                if count < 6500:
                    if line == '\n':
                        w1.write(line)
                        count += 1
                    else:
                        w1.write('\t'.join(line.strip().split('\t')[::-1]) + '\n')
                else:
                    if line == '\n':
                        w2.write(line)
                    else:
                        w2.write('\t'.join(line.strip().split('\t')[::-1]) + '\n')

In [5]:
# load the data again

data = []
labels = []
with open('trivia10k13train_updated.bio', 'r') as f:
    cur_data = []
    cur_label = []
    for line in f.readlines():
        if line == '\n':
            data.append(cur_data)
            labels.append(cur_label)
            cur_data = []
            cur_label = []
        else:
            cur_data.append(line.strip().split('\t')[0])
            cur_label.append(line.strip().split('\t')[1])

In [6]:
for w, e in zip(data[0], labels[0]):
    print(f'{w}\t{e}')


steve	B-Actor
mcqueen	I-Actor
provided	O
a	O
thrilling	B-Plot
motorcycle	I-Plot
chase	I-Plot
in	I-Plot
this	I-Plot
greatest	B-Opinion
of	I-Opinion
all	I-Opinion
ww	B-Plot
2	I-Plot
prison	I-Plot
escape	I-Plot
movies	I-Plot


### Rule-based

In [7]:
predictions = []
for word in data[0]:
    if word == 'steve':
        predictions.append('B-Actor')
    else:
        predictions.append('O')

In [8]:
f1_score(labels[0], predictions, average='weighted')

0.08496732026143791

### sklearn_crfsuite

In [9]:
df = pd.DataFrame({'sent_id': [i for j in [[i] * len(s) for i, s in enumerate(data)] for i in j],
                   'data': [i for j in data for i in j],
                   'entities': [i for j in labels for i in j]})
df.head(10)

,sent_id,data,entities
0,0,steve,B-Actor
1,0,mcqueen,I-Actor
2,0,provided,O
3,0,a,O
4,0,thrilling,B-Plot
5,0,motorcycle,I-Plot
6,0,chase,I-Plot
7,0,in,I-Plot
8,0,this,I-Plot
9,0,greatest,B-Opinion


In [10]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s['data'].values.tolist(), 
                                                           s['entities'].values.tolist())]
        self.grouped = self.data.groupby('sent_id').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

getter = SentenceGetter(df)
sentences = getter.sentences

In [11]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit()
    }
    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper()
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper()
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [12]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [13]:
X[0][:2]

[{'bias': 1.0,
  'word.lower()': 'steve',
  'word[-3:]': 'eve',
  'word[-2:]': 've',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  'BOS': True,
  '+1:word.lower()': 'mcqueen',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False},
 {'bias': 1.0,
  'word.lower()': 'mcqueen',
  'word[-3:]': 'een',
  'word[-2:]': 'en',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  '-1:word.lower()': 'steve',
  '-1:word.istitle()': False,
  '-1:word.isupper()': False,
  '+1:word.lower()': 'provided',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False}]

In [14]:
X_train = X[:6500]
X_test = X[6500:]
y_train = y[:6500]
y_test = y[6500:]

In [15]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
    verbose=True
)
crf.fit(X_train, y_train)

loading training data to CRFsuite: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6500/6500 [00:01<00:00, 5323.50it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 61139
Seconds required: 0.349

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.60  loss=338480.23 active=60235 feature_norm=1.00
Iter 2   time=0.89  loss=297824.58 active=59652 feature_norm=6.29
Iter 3   time=0.58  loss=232374.69 active=58689 feature_norm=6.13
Iter 4   time=0.61  loss=226916.98 active=59841 feature_norm=5.73
Iter 5   time=0.58  loss=199322.57 active=60030 feature_norm=5.34
Iter 6   time=0.90  loss=193155.61 active=60511 feature_norm=5.16
Iter 7   time=0.91  loss=187596.58 active=60520 feature_norm=4.95
Iter 8   time=0.87  loss=181148.76 active=60629 feature_norm=4.77
Iter 9   time=0.59  loss=177449.91 active=60152 feature_norm=4.56
Iter 

D:\Programs\anaconda3\envs\dl2\lib\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100, verbose=True)

In [16]:
all_entities = sorted(df.entities.unique().tolist())

In [17]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=all_entities)

0.8428243759486276

In [18]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=[i for i in all_entities if i != 'O'])

0.8360403580025992

In [19]:
print(metrics.flat_classification_report(y_test, y_pred, labels = all_entities))

D:\Programs\anaconda3\envs\dl2\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=['B-Actor', 'B-Award', 'B-Character_Name', 'B-Director', 'B-Genre', 'B-Opinion', 'B-Origin', 'B-Plot', 'B-Quote', 'B-Relationship', 'B-Soundtrack', 'B-Year', 'I-Actor', 'I-Award', 'I-Character_Name', 'I-Director', 'I-Genre', 'I-Opinion', 'I-Origin', 'I-Plot', 'I-Quote', 'I-Relationship', 'I-Soundtrack', 'I-Year', 'O'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


                  precision    recall  f1-score   support

         B-Actor       0.94      0.93      0.94      1050
         B-Award       0.76      0.67      0.72        58
B-Character_Name       0.75      0.36      0.48       165
      B-Director       0.91      0.81      0.85       414
         B-Genre       0.88      0.83      0.85       663
       B-Opinion       0.51      0.33      0.40       114
        B-Origin       0.58      0.49      0.53       127
          B-Plot       0.46      0.44      0.45      1022
         B-Quote       0.67      0.21      0.32        19
  B-Relationship       0.69      0.47      0.56        93
    B-Soundtrack       0.33      0.09      0.14        11
          B-Year       0.98      0.98      0.98       584
         I-Actor       0.96      0.91      0.93      1504
         I-Award       0.86      0.75      0.80       173
I-Character_Name       0.80      0.36      0.49       121
      I-Director       0.93      0.81      0.86       402
         I-Ge

In [20]:
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
B-Soundtrack -> I-Soundtrack 7.111965
B-Actor -> I-Actor 6.863916
I-Soundtrack -> I-Soundtrack 6.237996
I-Quote -> I-Quote 6.169556
B-Quote -> I-Quote 5.993501
B-Director -> I-Director 5.922611
B-Character_Name -> I-Character_Name 5.754599
I-Relationship -> I-Relationship 5.364200
B-Plot -> I-Plot  5.289407
B-Award -> I-Award 5.165195
B-Relationship -> I-Relationship 5.103005
B-Origin -> I-Origin 5.102667
I-Plot -> I-Plot  4.971816
I-Origin -> I-Origin 4.760625
I-Year -> I-Year  4.736332
I-Opinion -> I-Opinion 4.657728
I-Character_Name -> I-Character_Name 4.614719
I-Award -> I-Award 4.600034
I-Actor -> I-Actor 4.522861
I-Genre -> I-Genre 4.450081

Top unlikely transitions:
I-Genre -> B-Genre -3.570867
I-Plot -> I-Relationship -3.606019
O      -> I-Quote -3.630814
I-Plot -> I-Actor -3.723168
I-Plot -> I-Genre -3.733947
I-Origin -> I-Plot  -3.849100
O      -> I-Award -3.904364
B-Year -> I-Genre -3.996390
I-Plot -> I-Opinion -4.067416
O      -> I-Actor -4.129823
I-

In [21]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
6.484654 B-Year   word.isdigit()
5.840164 B-Quote  -1:word.lower():line
5.444134 B-Actor  word.lower():cher
5.219260 O        word.lower():starring
5.094407 O        word.lower():directed
4.915390 B-Genre  word.lower():animated
4.865797 B-Soundtrack -1:word.lower():song
4.827272 O        word.lower():stars
4.685116 B-Genre  word.lower():thriller
4.476266 B-Genre  word.lower():slasher
4.398568 B-Opinion -1:word.lower():films
4.360997 B-Genre  word.lower():crime
4.272314 B-Plot   -1:word.lower():goers
4.232554 B-Award  word.lower():oscars
4.209874 I-Plot   +1:word.lower():romeo
4.167938 B-Award  +1:word.lower():cannes
4.139843 B-Origin word.lower():shakespearean
4.139154 B-Director word.lower():scorsese
4.131031 B-Year   word.lower():nineties
4.104167 B-Genre  word.lower():animation
4.047133 I-Director +1:word.lower():wrote
4.006794 B-Origin word.lower():shakespeare
3.951604 B-Origin +1:word.lower():sultry
3.947418 B-Actor  -1:word.lower():starring
3.946435 B-Actor  BOS
3.9

In [22]:
eli5.show_weights(crf, top=10)

From \ To,O,B-Actor,I-Actor,B-Award,I-Award,B-Character_Name,I-Character_Name,B-Director,I-Director,B-Genre,I-Genre,B-Opinion,I-Opinion,B-Origin,I-Origin,B-Plot,I-Plot,B-Quote,I-Quote,B-Relationship,I-Relationship,B-Soundtrack,I-Soundtrack,B-Year,I-Year
O,3.042,1.555,-4.13,1.273,-3.904,1.497,-3.335,1.635,-3.357,1.298,-4.455,0.881,-4.748,1.393,-4.832,1.388,-6.556,1.326,-3.631,1.042,-4.594,1.928,-2.769,0.82,-3.459
B-Actor,-0.244,-1.65,6.864,-0.148,-0.684,-1.175,-0.935,-0.829,-1.24,-0.452,-1.072,-1.05,-1.059,-0.869,-1.27,-0.205,-2.647,-0.314,-0.605,-0.559,-0.592,0.0,-0.136,-0.671,-0.185
I-Actor,1.622,-4.289,4.523,-0.014,-1.759,-1.827,-1.536,-1.677,-1.545,0.436,-1.696,0.213,-1.99,-0.882,-2.298,0.935,-4.546,-1.252,-1.786,-1.454,-1.49,-0.083,-0.72,-1.018,-1.462
B-Award,0.297,-0.637,-0.224,-1.224,5.165,-0.171,0.0,-0.259,-0.03,-0.891,-0.493,-1.039,-0.485,-0.24,-0.453,-1.361,-1.438,0.0,0.0,-0.264,-0.19,0.0,0.0,0.001,0.0
I-Award,0.152,-0.213,-0.633,-2.309,4.6,-0.803,-0.311,-0.254,-0.533,-0.129,-1.095,-0.101,-1.097,0.0,-1.121,-1.585,-2.292,-0.031,-0.354,0.355,-1.163,0.038,-0.258,-0.392,-0.093
B-Character_Name,0.445,-0.855,-1.474,-0.234,-0.63,-1.5,5.755,-1.007,-1.013,-1.461,-1.069,-1.15,-1.263,-1.109,-1.61,1.588,-3.23,-0.759,-0.795,-0.916,-1.057,0.0,-0.247,-1.18,-0.584
I-Character_Name,0.301,-1.069,-1.333,-0.412,-0.722,-2.345,4.615,-0.928,-0.673,-1.207,-0.857,-0.15,-1.293,-0.859,-1.457,1.343,-3.449,-0.137,-0.878,0.0,-1.083,0.0,-0.324,-0.686,-0.621
B-Director,0.014,-1.58,-1.803,-0.248,-1.182,-0.392,-1.057,-1.963,5.923,0.505,-2.691,0.049,-1.547,-0.347,-1.713,-2.728,-2.553,-0.344,-0.664,0.362,-1.237,0.0,-0.277,-0.605,-0.788
I-Director,1.591,-1.021,-1.457,-0.443,-0.903,-1.036,-0.664,-1.663,4.212,1.791,-1.9,0.493,-1.331,0.196,-1.681,0.06,-2.645,-0.236,-0.459,1.264,-1.08,0.0,-0.14,0.488,-0.954
B-Genre,0.553,-0.484,-1.793,-0.03,-2.087,-2.018,-1.414,0.292,-1.894,-4.583,3.965,0.546,-2.673,0.297,-2.811,-0.311,-3.438,-0.807,-1.065,0.421,-2.379,-0.386,-0.877,-0.284,-1.668


### Spacy

In [23]:
!python -m spacy init config base_config.cfg -p ner


[x] The provided output file already exists. To force overwriting the config
file, set the --force or -F flag.



In [24]:
!python -m spacy init fill-config base_config.cfg config.cfg

[!] Nothing to auto-fill: base config is already complete
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [25]:
!python -m spacy convert restauranttrain_updated.bio . -t json -c ner

[i] Auto-detected token-per-line NER format
[i] Grouping every 1 sentences into a document.
[!] To generate better training data, you may want to group sentences into
documents with `-n 10`.
[+] Generated output file (1 documents): restauranttrain_updated.json


In [26]:
!python -m spacy convert restauranttrain_updated_train.bio . -t json -c ner

[i] Auto-detected token-per-line NER format
[i] Grouping every 1 sentences into a document.
[!] To generate better training data, you may want to group sentences into
documents with `-n 10`.
[+] Generated output file (1 documents): restauranttrain_updated_train.json


In [27]:
!python -m spacy convert restauranttrain_updated_valid.bio . -t json -c ner

[i] Auto-detected token-per-line NER format
[i] Grouping every 1 sentences into a document.
[!] To generate better training data, you may want to group sentences into
documents with `-n 10`.
[+] Generated output file (1 documents): restauranttrain_updated_valid.json


In [28]:
#!python -m spacy train config.cfg --output ./output --paths.train spacy_data/restauranttrain_updated_train.spacy --paths.dev spacy_data/restauranttrain_updated_valid.spacy

In [29]:
!python -m spacy evaluate output/model-last spacy_data/restauranttrain_updated_valid.spacy

[i] Using CPU

================================== Results ==================================

TOK     100.00
NER P   78.97 
NER R   76.79 
NER F   77.86 
SPEED   22517 


=============================== NER (per type) ===============================

                      P       R       F
Rating            74.89   75.58   75.23
Cuisine           79.19   82.76   80.93
Dish              70.50   71.62   71.05
Location          85.40   82.90   84.13
Hours             81.89   69.33   75.09
Price             88.98   85.61   87.26
Amenity           68.82   62.63   65.58
Restaurant_Name   84.62   77.00   80.63



In [30]:
nlp1 = spacy.load("output/model-best")
doc = nlp1(' '.join(data[2]))

spacy.displacy.render(doc, style="ent", jupyter=True)

### To biluo

In [31]:
with open('restauranttrain_updated.json', 'r') as f:
    d = json.load(f)

In [32]:
d[0]['paragraphs'][34]['sentences']

[{'tokens': [{'id': 0, 'orth': 'any', 'space': ' ', 'tag': '-', 'ner': 'O'},
   {'id': 1, 'orth': 'chance', 'space': ' ', 'tag': '-', 'ner': 'O'},
   {'id': 2, 'orth': 'i', 'space': ' ', 'tag': '-', 'ner': 'O'},
   {'id': 3, 'orth': 'can', 'space': ' ', 'tag': '-', 'ner': 'O'},
   {'id': 4, 'orth': 'get', 'space': ' ', 'tag': '-', 'ner': 'O'},
   {'id': 5, 'orth': 'a', 'space': ' ', 'tag': '-', 'ner': 'O'},
   {'id': 6,
    'orth': 'reservation',
    'space': ' ',
    'tag': '-',
    'ner': 'U-Amenity'},
   {'id': 7, 'orth': 'at', 'space': ' ', 'tag': '-', 'ner': 'O'},
   {'id': 8, 'orth': 'that', 'space': ' ', 'tag': '-', 'ner': 'O'},
   {'id': 9, 'orth': 'fine', 'space': ' ', 'tag': '-', 'ner': 'B-Amenity'},
   {'id': 10, 'orth': 'dining', 'space': ' ', 'tag': '-', 'ner': 'L-Amenity'},
   {'id': 11, 'orth': 'place', 'space': ' ', 'tag': '-', 'ner': 'O'},
   {'id': 12, 'orth': 'thats', 'space': ' ', 'tag': '-', 'ner': 'O'},
   {'id': 13, 'orth': 'the', 'space': ' ', 'tag': '-', 'ner':

In [33]:
nlp = spacy.load("en_core_web_sm")

In [34]:
tokens_dict = d[0]['paragraphs'][34]['sentences'][0]['tokens']
tokens = [i['orth'] for i in tokens_dict]

In [35]:
text = ' '.join(tokens)
doc = nlp(text)
entities = d[0]['paragraphs'][34]['entities']

In [36]:
text

'any chance i can get a reservation at that fine dining place thats the top rated on tripadvisor'

In [37]:
entities

[[23, 34, 'Amenity'], [43, 54, 'Amenity'], [71, 95, 'Rating']]

In [38]:
offsets_to_biluo_tags(doc, entities)

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'U-Amenity',
 'O',
 'O',
 'B-Amenity',
 'L-Amenity',
 'O',
 'O',
 'O',
 'O',
 'B-Rating',
 'I-Rating',
 'I-Rating',
 'L-Rating']

In [39]:
assert len(tokens) == len(offsets_to_biluo_tags(doc, entities)), 'Something went wrong'

AssertionError: Something went wrong

In [40]:
for ent in entities:
    t = text[ent[0]: ent[1]]
    print(f'Entity: {ent[2]}: {t}')

Entity: Amenity: reservation
Entity: Amenity: fine dining
Entity: Rating: top rated on tripadvisor


In [41]:
for k, m in zip(tokens, offsets_to_biluo_tags(doc, entities)):
    print(f'{k}\t{m}')

any	O
chance	O
i	O
can	O
get	O
a	O
reservation	U-Amenity
at	O
that	O
fine	B-Amenity
dining	L-Amenity
place	O
thats	O
the	O
top	O
rated	B-Rating
on	I-Rating
tripadvisor	I-Rating


In [42]:
from typing import List, Tuple, Union
def convert_to_biluo(text: str = '',
                     entities: List[Tuple] = None,
                     tokens: list = None,
                     missing: str = 'O') -> Tuple[Union[List[str], list, None], List[str]]:
    """
    Tokenize text and return text tokens and ner labels.

    Args:
        text: text
        entities: labels in spacy format
        tokens: already tokenized text, if you want it
        missing: lable for tokens without entities

    Returns:
        tokenized text and labels
    """

    # create dicts with start/end position of token and its index
    starts = []
    ends = []
    cur_index = 0
    tokens = text.split() if tokens is None else tokens

    for token in tokens:
        starts.append(cur_index)
        ends.append(cur_index + len(token))
        cur_index += len(token) + 1

    starts = {k: v for v, k in enumerate(starts)}
    ends = {k: v for v, k in enumerate(ends)}

    # this will be a list with token labels
    biluo = ["-" for _ in text.split()]

    # check that there are no overlapping entities
    entities_indexes = [list(range(i[0], i[1])) for i in entities]
    if max(Counter([i for j in entities_indexes for i in j]).values()) > 1:
        raise ValueError('You have overlapping entities')

    tokens_in_ents = {}

    # Handle entity cases
    for start_char, end_char, label in entities:
        for token_index in range(start_char, end_char):
            tokens_in_ents[token_index] = (start_char, end_char, label)
        start_token = starts.get(start_char)
        end_token = ends.get(end_char)
        # Only interested if the tokenization is correct
        if start_token is not None and end_token is not None:
            if start_token == end_token:
                biluo[start_token] = f"U-{label}"
            else:
                biluo[start_token] = f"B-{label}"
                for i in range(start_token + 1, end_token):
                    biluo[i] = f"I-{label}"
                biluo[end_token] = f"L-{label}"

    # put missing value for tokens without labels
    entity_chars = set()
    for start_char, end_char, label in entities:
        for i in range(start_char, end_char):
            entity_chars.add(i)

    for ind, token in enumerate(tokens):
        for i in range(list(starts.keys())[ind], list(ends.keys())[ind]):
            if i in entity_chars:
                break
        else:
            biluo[ind] = missing

    return tokens, biluo

In [43]:
# difference betwen my function and spacy's function
new_data = []
biluo_labels = []
for i in range(len(d[0]['paragraphs'])):
    tokens_dict = d[0]['paragraphs'][i]['sentences'][0]['tokens']
    tokens = [i['orth'] for i in tokens_dict]
    if len([i['orth'] for i in tokens_dict]) > 1:
        
        text = ' '.join(tokens)
        doc = nlp(text)
        entities = d[0]['paragraphs'][i]['entities']

        new_ents = offsets_to_biluo_tags(doc, entities)
        new_data.append(tokens)
        biluo_labels.append(new_ents)
        if len(tokens) != len(new_ents):
            
            ents2 = convert_to_biluo(text, entities)[1]
            print(i, entities, text)
            for ent in entities:
                t = text[ent[0]: ent[1]]
                print(f'Entity: {ent[2]}: {t}')
            for k, m, l in zip(tokens, new_ents, ents2):
                print(f'{k}\t{m}\t{l}')
            print()
        if i > 2000:
            break

34 [[23, 34, 'Amenity'], [43, 54, 'Amenity'], [71, 95, 'Rating']] any chance i can get a reservation at that fine dining place thats the top rated on tripadvisor
Entity: Amenity: reservation
Entity: Amenity: fine dining
Entity: Rating: top rated on tripadvisor
any	O	O
chance	O	O
i	O	O
can	O	O
get	O	O
a	O	O
reservation	U-Amenity	U-Amenity
at	O	O
that	O	O
fine	B-Amenity	B-Amenity
dining	L-Amenity	L-Amenity
place	O	O
thats	O	O
the	O	O
top	O	B-Rating
rated	B-Rating	I-Rating
on	I-Rating	I-Rating
tripadvisor	I-Rating	L-Rating

80 [[29, 39, 'Dish'], [46, 67, 'Location']] anyplace i can get some good pita bread thats not more than 4 miles out
Entity: Dish: pita bread
Entity: Location: not more than 4 miles
anyplace	O	O
i	O	O
can	O	O
get	O	O
some	O	O
good	O	O
pita	B-Dish	B-Dish
bread	L-Dish	L-Dish
thats	O	O
not	O	B-Location
more	B-Location	I-Location
than	I-Location	I-Location
4	I-Location	I-Location
miles	I-Location	L-Location
out	L-Location	O

84 [[15, 26, 'Amenity']] anywhere thats not crowd

1018 [[18, 24, 'Cuisine'], [31, 39, 'Location'], [45, 49, 'Amenity'], [50, 59, 'Cuisine']] could you find me burger place close by that isnt fast food
Entity: Cuisine: burger
Entity: Location: close by
Entity: Amenity: isnt
Entity: Cuisine: fast food
could	O	O
you	O	O
find	O	O
me	O	O
burger	U-Cuisine	U-Cuisine
place	O	O
close	B-Location	B-Location
by	L-Location	L-Location
that	O	O
isnt	B-Amenity	U-Amenity
fast	L-Amenity	B-Cuisine
food	B-Cuisine	L-Cuisine

1214 [[19, 23, 'Rating'], [24, 39, 'Amenity'], [40, 45, 'Cuisine'], [46, 53, 'Amenity'], [59, 78, 'Price']] do you know of any good all you can eat sushi buffets that arent too expensive
Entity: Rating: good
Entity: Amenity: all you can eat
Entity: Cuisine: sushi
Entity: Amenity: buffets
Entity: Price: arent too expensive
do	O	O
you	O	O
know	O	O
of	O	O
any	O	O
good	U-Rating	U-Rating
all	B-Amenity	B-Amenity
you	I-Amenity	I-Amenity
can	I-Amenity	I-Amenity
eat	L-Amenity	L-Amenity
sushi	U-Cuisine	U-Cuisine
buffets	U-Amenity	U-Amenity
that

In [44]:
# convert the data
%%time
new_data = []
biluo_labels = []
for i in range(len(d[0]['paragraphs'])):
    tokens_dict = d[0]['paragraphs'][i]['sentences'][0]['tokens']
    tokens = [i['orth'] for i in tokens_dict]
    if len([i['orth'] for i in tokens_dict]) > 1:
        
        text = ' '.join(tokens)
        doc = nlp(text)
        entities = d[0]['paragraphs'][i]['entities']

        new_ents = offsets_to_biluo_tags(doc, entities)
        if entities == []:
            new_ents = ['O'] * len(tokens)
        new_data.append(tokens)
        
        biluo_labels.append(new_ents)
        if len(tokens) != len(new_ents):
            
            ents2 = convert_to_biluo(text, entities)[1]
            biluo_labels[-1] = ents2


Wall time: 39.8 s


In [45]:
len(new_data)

7634

In [46]:
df = pd.DataFrame({'sent_id': [i for j in [[i] * len(s) for i, s in enumerate(new_data)] for i in j],
                   'data': [i for j in new_data for i in j],
                   'entities': [i for j in biluo_labels for i in j]})
df.head()

,sent_id,data,entities
0,0,2,B-Rating
1,0,start,L-Rating
2,0,restaurants,O
3,0,with,O
4,0,inside,B-Amenity


In [47]:
getter = SentenceGetter(df)
sentences = getter.sentences
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [48]:
X_train = X[:6500]
X_test = X[6500:]
y_train = y[:6500]
y_test = y[6500:]

In [49]:

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

D:\Programs\anaconda3\envs\dl2\lib\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [50]:
all_entities = sorted(df.entities.unique().tolist())

In [51]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=all_entities)

0.8860368692111157

In [52]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=[i for i in all_entities if i != 'O'])

0.7861251286223245

In [53]:
print(metrics.flat_classification_report(y_test, y_pred, labels = all_entities))

                   precision    recall  f1-score   support

        B-Amenity       0.69      0.71      0.70       192
        B-Cuisine       0.77      0.76      0.77        71
           B-Dish       0.74      0.76      0.75       132
          B-Hours       0.86      0.83      0.85        99
       B-Location       0.77      0.76      0.76       214
          B-Price       0.87      0.57      0.68        23
         B-Rating       0.67      0.71      0.69        58
B-Restaurant_Name       0.80      0.79      0.80       123
        I-Amenity       0.44      0.44      0.44        94
        I-Cuisine       0.58      0.64      0.61        11
           I-Dish       0.71      0.51      0.59        57
          I-Hours       0.89      0.79      0.84        86
       I-Location       0.68      0.72      0.70       139
          I-Price       1.00      0.56      0.71         9
         I-Rating       0.20      0.10      0.13        10
I-Restaurant_Name       0.73      0.77      0.75       

D:\Programs\anaconda3\envs\dl2\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=['B-Amenity', 'B-Cuisine', 'B-Dish', 'B-Hours', 'B-Location', 'B-Price', 'B-Rating', 'B-Restaurant_Name', 'I-Amenity', 'I-Cuisine', 'I-Dish', 'I-Hours', 'I-Location', 'I-Price', 'I-Rating', 'I-Restaurant_Name', 'L-Amenity', 'L-Cuisine', 'L-Dish', 'L-Hours', 'L-Location', 'L-Price', 'L-Rating', 'L-Restaurant_Name', 'O', 'U-Amenity', 'U-Cuisine', 'U-Dish', 'U-Hours', 'U-Location', 'U-Price', 'U-Rating', 'U-Restaurant_Name'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [54]:
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
B-Dish -> L-Dish  8.357350
B-Restaurant_Name -> L-Restaurant_Name 8.232111
B-Cuisine -> L-Cuisine 8.133037
I-Restaurant_Name -> L-Restaurant_Name 7.984668
I-Location -> L-Location 7.866484
B-Restaurant_Name -> I-Restaurant_Name 7.777496
B-Dish -> I-Dish  7.691174
B-Amenity -> L-Amenity 7.646804
I-Dish -> L-Dish  7.599795
I-Amenity -> L-Amenity 7.559043
B-Location -> I-Location 7.545233
I-Location -> I-Location 7.507271
B-Price -> L-Price 7.449166
B-Location -> L-Location 7.448815
B-Hours -> L-Hours 7.414274
B-Rating -> L-Rating 7.288900
I-Dish -> I-Dish  7.196051
B-Cuisine -> I-Cuisine 7.186808
I-Cuisine -> L-Cuisine 7.144503
B-Hours -> I-Hours 7.109238

Top unlikely transitions:
O      -> I-Price -2.481557
U-Cuisine -> B-Cuisine -2.482749
B-Restaurant_Name -> O       -2.500061
O      -> L-Price -2.565038
O      -> I-Rating -2.642450
I-Location -> O       -2.652334
I-Hours -> O       -2.653665
B-Location -> O       -2.654625
U-Cuisine -> U-Cuisine -2.673171
U-Ho

In [55]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
6.434466 U-Location word.lower():closest
5.878651 U-Price  +1:word.lower():prices
5.832064 U-Price  +1:word.lower():priced
5.815888 U-Cuisine word.lower():seafood
5.789389 U-Price  +1:word.lower():price
5.627061 U-Location word.lower():nearest
5.544316 U-Restaurant_Name word.lower():subway
5.291834 U-Price  word.lower():cheapest
4.931155 U-Price  word.lower():expensive
4.832626 U-Rating word.lower():best
4.703637 O        word.lower():and
4.685988 U-Price  word.lower():affordable
4.617837 U-Restaurant_Name -1:word.lower():does
4.597654 O        word.lower():which
4.495425 O        word.lower():some
4.414698 U-Location -1:word.lower():in
4.396757 O        word.lower():this
4.382073 U-Rating word.lower():great
4.339903 U-Amenity word.lower():takeout
4.330952 U-Location word.lower():downtown
4.314481 U-Amenity word.lower():delivers
4.244998 L-Price  +1:word.lower():price
4.229204 B-Amenity +1:word.lower():go
4.207747 U-Cuisine word.lower():diners
4.193463 U-Amenity word.lowe

In [56]:
eli5.show_weights(crf, top=10)

From \ To,O,B-Amenity,I-Amenity,L-Amenity,U-Amenity,B-Cuisine,I-Cuisine,L-Cuisine,U-Cuisine,B-Dish,I-Dish,L-Dish,U-Dish,B-Hours,I-Hours,L-Hours,U-Hours,B-Location,I-Location,L-Location,U-Location,B-Price,I-Price,L-Price,U-Price,B-Rating,I-Rating,L-Rating,U-Rating,B-Restaurant_Name,I-Restaurant_Name,L-Restaurant_Name,U-Restaurant_Name
O,3.046,2.248,-3.968,-3.01,1.984,1.168,-2.12,-2.243,1.814,1.829,-2.243,-2.201,1.753,1.742,-2.803,-2.457,1.319,2.258,-3.223,-2.892,1.974,1.519,-2.482,-2.565,1.782,2.099,-2.642,-2.167,2.283,2.694,-3.076,-2.305,2.007
B-Amenity,-2.859,-1.229,6.772,7.647,-0.976,-0.703,0.0,0.0,-1.223,-0.498,0.0,0.0,-0.719,-0.618,0.0,0.0,-0.697,-1.069,-0.081,-0.123,-0.769,-0.0,0.0,0.0,-0.061,-0.111,0.0,0.0,-0.049,-0.011,-0.013,0.0,-0.125
I-Amenity,-2.902,-0.833,6.796,7.559,-0.988,-0.706,0.0,0.0,-0.993,-0.251,0.0,0.0,-0.627,-0.58,0.0,0.0,-0.392,-1.491,-0.119,-0.033,-1.039,-0.0,0.0,0.0,-0.04,0.0,0.0,0.0,0.0,-0.224,0.0,0.0,-0.218
L-Amenity,0.933,-2.365,-2.256,-1.767,-2.191,0.759,-0.678,-0.784,0.925,0.081,-0.613,-0.822,0.423,0.66,-0.807,-0.639,0.249,0.101,-1.292,-1.376,0.515,0.546,-0.888,-0.577,0.167,0.006,-0.328,-0.681,-0.54,0.487,-1.167,-0.997,0.75
U-Amenity,1.29,-2.187,-1.755,-1.509,-1.537,1.078,-0.511,-0.827,1.366,0.438,-0.193,-0.338,0.517,1.025,-0.47,-0.255,1.35,1.112,-0.68,-0.947,0.364,0.0,-0.383,-0.312,1.395,-0.338,-0.174,-0.337,1.326,0.651,-0.688,-0.539,1.11
B-Cuisine,-2.25,-0.428,0.0,-0.236,-0.433,-0.58,7.187,8.133,-0.975,0.0,0.0,0.0,-0.193,-0.037,0.0,0.0,-0.18,-0.546,0.0,0.0,-0.201,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-Cuisine,-1.854,0.0,0.0,0.0,0.0,-0.179,6.069,7.145,-0.88,0.0,0.0,0.0,-0.417,0.0,0.0,0.0,-0.062,-1.009,0.0,0.0,-0.292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
L-Cuisine,1.815,0.0,-0.526,-0.455,-0.66,-0.177,0.0,-0.79,-1.321,0.0,0.0,-0.015,1.58,-0.379,0.0,-0.18,0.0,0.504,-0.459,-0.376,0.531,0.0,0.0,-0.262,0.356,0.0,0.0,0.0,0.983,0.0,-0.134,-0.324,-0.323
U-Cuisine,1.898,0.953,-1.19,-1.583,1.481,-2.483,-0.539,-1.496,-2.673,-0.923,-0.208,-0.696,1.583,0.508,-0.554,-0.435,0.188,1.399,-0.792,-0.886,0.943,-0.133,-0.149,-0.568,0.013,-0.283,-0.108,-0.589,1.04,-0.351,-0.841,-1.314,0.666
B-Dish,-2.067,-0.292,0.0,0.0,-0.216,-0.113,0.0,0.0,-0.727,-0.267,7.691,8.357,-0.7,-0.11,0.0,0.0,-0.09,-0.471,0.0,0.0,-0.241,0.0,0.0,0.0,0.0,-0.015,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Bi-LSTM

In [57]:
import json
from collections import Counter
from tqdm.notebook import tqdm
import joblib
from typing import List, Tuple, Union, Dict

In [58]:
def get_word_to_idx(count: List[Tuple[str, int]],
                   min_words: Union[int, float] = 0.0,
                   max_words: Union[int, float] = 1.0) -> Dict[str, int]:
    max_count = count[0][1]
    if isinstance(min_words, float):
        min_words = max_count * min_words
    if isinstance(max_words, float):
        max_words = max_count * max_words
        
    all_words = [w[0] for w in count if max_words >= w[1] >= min_words]
    
    all_words = ['<pad>', '<unk>'] + all_words
    
    word_to_idx = {k: v for k, v in zip(all_words, range(0, len(all_words)))}
    return word_to_idx

In [59]:
count = Counter([i for j in new_data for i in j])
word_to_idx = get_word_to_idx(count.most_common(), min_words=1)
len(word_to_idx)

3805

In [60]:
tags = sorted(list({i for j in biluo_labels for i in j}))
tags.remove('O')

tag_to_idx = {}
for ind, entity in enumerate(tags):
    tag_to_idx[f'{entity}'] = len(tag_to_idx)

for special_tag in ['O', 'PAD']:
    tag_to_idx[special_tag] = len(tag_to_idx)
    
tag_to_idx

{'B-Amenity': 0,
 'B-Cuisine': 1,
 'B-Dish': 2,
 'B-Hours': 3,
 'B-Location': 4,
 'B-Price': 5,
 'B-Rating': 6,
 'B-Restaurant_Name': 7,
 'I-Amenity': 8,
 'I-Cuisine': 9,
 'I-Dish': 10,
 'I-Hours': 11,
 'I-Location': 12,
 'I-Price': 13,
 'I-Rating': 14,
 'I-Restaurant_Name': 15,
 'L-Amenity': 16,
 'L-Cuisine': 17,
 'L-Dish': 18,
 'L-Hours': 19,
 'L-Location': 20,
 'L-Price': 21,
 'L-Rating': 22,
 'L-Restaurant_Name': 23,
 'U-Amenity': 24,
 'U-Cuisine': 25,
 'U-Dish': 26,
 'U-Hours': 27,
 'U-Location': 28,
 'U-Price': 29,
 'U-Rating': 30,
 'U-Restaurant_Name': 31,
 'O': 32,
 'PAD': 33}

In [61]:
from typing import Optional
def pad_sequences(
    sequences: List,
    maxlen: Optional[int],
    dtype: str = 'int32',
    padding: str = 'post',
    truncating: str = 'post',
    value: int = 0,
) -> np.array:
    """Pad sequences to the same length.
    from Keras

    This function transforms a list of
    `num_samples` sequences (lists of integers)
    into a 2D Numpy array of shape `(num_samples, num_timesteps)`.
    `num_timesteps` is either the `maxlen` argument if provided,
    or the length of the longest sequence otherwise.

    Sequences that are shorter than `num_timesteps`
    are padded with `value` at the end.

    Sequences longer than `num_timesteps` are truncated
    so that they fit the desired length.
    The position where padding or truncation happens is determined by
    the arguments `padding` and `truncating`, respectively.

    Pre-padding is the default.

    # Arguments
        sequences: List of lists, where each element is a sequence.
        maxlen: Int, maximum length of all sequences.
        dtype: Type of the output sequences.
            To pad sequences with variable length strings, you can use `object`.
        padding: String, 'pre' or 'post':
            pad either before or after each sequence.
        truncating: String, 'pre' or 'post':
            remove values from sequences larger than
            `maxlen`, either at the beginning or at the end of the sequences.
        value: Float or String, padding value.

    # Returns
        x: Numpy array with shape `(len(sequences), maxlen)`

    # Raises
        ValueError: In case of invalid values for `truncating` or `padding`,
            or in case of invalid shape for a `sequences` entry.
    """
    if not hasattr(sequences, '__len__'):
        raise ValueError('`sequences` must be iterable.')
    num_samples = len(sequences)

    lengths = []
    for x in sequences:
        try:
            lengths.append(len(x))
        except TypeError:
            raise ValueError('`sequences` must be a list of iterables. ' 'Found non-iterable: ' + str(x))

    if maxlen is None:
        maxlen = np.max(lengths)

    # take the sample shape from the first non empty sequence
    # checking for consistency in the main loop below.
    sample_shape = ()
    for s in sequences:
        if len(s) > 0:
            sample_shape = np.asarray(s).shape[1:]
            break

    x = np.full((num_samples, maxlen) + sample_shape, value, dtype=dtype)
    for idx, s in enumerate(sequences):
        if not len(s):
            continue  # empty list/array was found
        if truncating == 'pre':
            trunc = s[-maxlen:]
        elif truncating == 'post':
            trunc = s[:maxlen]
        else:
            raise ValueError(f'Truncating type "{truncating}" ' 'not understood')

        # check `trunc` has expected shape
        trunc = np.asarray(trunc, dtype=dtype)
        if trunc.shape[1:] != sample_shape:
            raise ValueError(
                f'Shape of sample {trunc.shape[1:]} of sequence at position {idx}'
                f'is different from expected shape {sample_shape}'
            )

        if padding == 'post':
            x[idx, : len(trunc)] = trunc
        elif padding == 'pre':
            x[idx, -len(trunc) :] = trunc
        else:
            raise ValueError(f'Padding type "{padding}" not understood')
    return x


In [62]:
from torch.utils.data import Dataset
class NerDataset(Dataset):
    def __init__(self, ner_data: List, ner_tags: List, word_to_idx: Dict, tag_to_idx: Dict, **kwarg: Dict):
        self.ner_data = ner_data
        self.ner_tags = ner_tags
        self.word_to_idx = word_to_idx
        self.tag_to_idx = tag_to_idx
        
    def __len__(self) -> int:
        return len(self.ner_data)
    
    def __getitem__(self, idx: int) -> Tuple[np.array, int, np.array]:
        line = self.ner_data[idx]
        
        tokens = [self.word_to_idx[w] if w in self.word_to_idx else self.word_to_idx['<unk>'] for w in self.ner_data[idx]]
        
        labels = [self.tag_to_idx[w] for w in self.ner_tags[idx]]
        
        return np.array(tokens), len(tokens), np.array(labels)

In [63]:
X_train = new_data[:6500]
X_test = new_data[6500:]
y_train = biluo_labels[:6500]
y_test = biluo_labels[6500:]

train_dataset = NerDataset(X_train, y_train, word_to_idx, tag_to_idx)
valid_dataset = NerDataset(X_test, y_test, word_to_idx, tag_to_idx)

In [64]:
train_dataset.__getitem__(5)

(array([   2,    4,   11,    6,   29,   12, 1060]),
 7,
 array([32, 32, 32, 32, 30,  0, 16]))

In [65]:
class Collator:
    def __init__(self, test=False, percentile=100, pad_value=0):
        self.test = test
        self.percentile = percentile
        self.pad_value = pad_value

    def __call__(self, batch):
        tokens, lens, labels = zip(*batch)
        lens = np.array(lens)

        max_len = min(int(np.percentile(lens, self.percentile)), 100)

        tokens = torch.tensor(
            pad_sequences(tokens, maxlen=max_len, padding='post', value=self.pad_value), dtype=torch.long
        )
        lens = torch.tensor([min(i, max_len) for i in lens], dtype=torch.long)
        labels = torch.tensor(
            pad_sequences(labels, maxlen=max_len, padding='post', value=self.pad_value), dtype=torch.long
        )

        return tokens, lens, labels

In [66]:
import torch
collator = Collator(percentile=100, pad_value=tag_to_idx['PAD'])
train_loader = torch.utils.data.DataLoader(
            train_dataset,
            batch_size=128,
            num_workers=0,
            collate_fn=collator,
            shuffle=False,
        )

valid_loader = torch.utils.data.DataLoader(
            valid_dataset,
            batch_size=64,
            num_workers=0,
            collate_fn=collator,
            shuffle=False,
        )


In [67]:
for batch in train_loader:
    break

In [68]:
batch

(tensor([[ 142,  713,   22,  ...,   33,   33,   33],
         [  66,   93,  657,  ...,   33,   33,   33],
         [1310,  658,  659,  ...,   33,   33,   33],
         ...,
         [  17,   14,   23,  ...,   33,   33,   33],
         [  17,   14,   23,  ...,   33,   33,   33],
         [  17,   14,   23,  ...,   33,   33,   33]]),
 tensor([ 6,  6,  6, 12,  8,  7,  6,  6,  5,  6, 14, 16,  6,  3, 12,  5, 12, 13,
         14,  5, 11, 10,  4,  2,  5, 10,  7,  5,  9,  7,  6,  5,  7, 18,  5,  8,
          4,  6,  5,  4,  4,  7,  5,  3,  6,  5,  9,  6,  6,  3,  5,  6,  6,  8,
          9, 13,  8, 10,  8,  6,  9,  7, 15,  8, 11,  6,  8, 10,  8,  9,  7,  4,
         11,  8, 10,  6,  6,  4,  6, 15,  7,  4, 10, 13,  5,  4,  7,  9,  6,  9,
          5, 21,  8,  6,  8,  8,  4,  9, 10,  8,  8,  7, 10,  7, 12,  8, 11,  7,
         10,  7, 10,  7, 11, 14, 17, 11,  7,  7, 10,  9, 12,  8,  9,  7,  6,  8,
         10, 10]),
 tensor([[ 6, 22, 32,  ..., 33, 33, 33],
         [ 6, 22, 32,  ..., 33, 33, 33]

In [69]:
batch[0].shape, batch[1].shape, batch[2].shape

(torch.Size([128, 21]), torch.Size([128]), torch.Size([128, 21]))

In [70]:
import fasttext.util
# fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')

In [71]:
fasttext.util.reduce_model(ft, 100)

In [72]:
ft['Dog']

array([-2.30683208e-01,  4.85748649e-01,  2.43170276e-01, -6.06558442e-01,
       -7.22950697e-03,  2.40569711e-02,  3.73850733e-01, -1.90829784e-02,
       -8.93123001e-02,  3.20367455e-01,  4.09503758e-01,  1.37113646e-01,
        1.45256609e-01,  1.75826222e-01, -3.85210328e-02, -4.15239483e-01,
        1.05385855e-01,  3.26399505e-02, -5.99754080e-02,  3.06701660e-02,
       -5.43401837e-02, -2.32265443e-01,  4.72719222e-02, -2.09649265e-01,
       -9.31737795e-02, -1.52251065e-01, -9.48294550e-02, -1.46978915e-01,
        2.65109241e-01, -3.80678624e-02,  3.70465145e-02, -1.84240699e-01,
       -2.94598877e-01,  1.87209398e-01, -7.89878517e-03, -9.70751494e-02,
        9.30286944e-04,  7.12471902e-02,  4.45247479e-02, -9.17917490e-02,
       -2.23052889e-01, -1.23794161e-01, -2.95656294e-01, -1.72065645e-01,
        1.74785703e-02,  3.35660845e-01,  1.52157724e-01,  2.75652409e-01,
       -7.62667581e-02, -2.74237543e-02,  2.23682314e-01, -7.20107928e-02,
       -1.49004787e-01, -

In [73]:
def build_matrix(
    word_dict: Dict,
    embedding_index,
    max_features: int = 100000,
    embed_size: int = 100,
) -> Tuple[np.array, int, List]:
    """
    Create embedding matrix

    Args:
        word_dict: tokenizer
        embedding_index: Fasttext embeddings
        max_features: max features to use
        embed_size: size of embeddings

    Returns:
        embedding matrix, number of of words and the list of not found words
    """
    embedding_index = ft
    nb_words = min(max_features, len(word_dict))
    embedding_matrix = np.zeros((nb_words, embed_size))

    for word, i in word_dict.items():
        embedding_matrix[i] = embedding_index[word]
    return embedding_matrix, nb_words

In [74]:
embedding_matrix, nb_words = build_matrix(word_dict=word_to_idx, embedding_index=ft)

In [75]:
from torch import nn

In [76]:
embedding_matrix = torch.tensor(embedding_matrix, dtype=torch.float32)
embedding = nn.Embedding.from_pretrained(embedding_matrix)
embedding.weight.requires_grad = False

In [77]:
import torch.functional as F
from torch.nn.parameter import Parameter
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from typing import Any

In [78]:
class SpatialDropout(nn.Module):
    """
    Spatial Dropout drops a certain percentage of dimensions from each word vector in the training sample
    implementation: https://discuss.pytorch.org/t/spatial-dropout-in-pytorch/21400
    explanation: https://www.kaggle.com/c/quora-insincere-questions-classification/discussion/76883
    """

    def __init__(self, p: float):
        super(SpatialDropout, self).__init__()
        self.spatial_dropout = nn.Dropout2d(p=p)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.permute(0, 2, 1)  # convert to [batch, channels, time]
        x = self.spatial_dropout(x)
        x = x.permute(0, 2, 1)  # back to [batch, time, channels]
        return x

In [79]:
class MultiHeadSelfAttention(nn.Module):
    """
    torch.nn.MultiHeadAttention wrapper to unify interface with other Attention classes
    Implementation of Dot-product Attention
    paper: https://arxiv.org/abs/1706.03762
    Time complexity: O(n^2)
    """

    def __init__(self, embed_dim: int, num_heads: int, dropout: float, **kwargs: Dict[str, Any]):
        super(MultiHeadSelfAttention, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, dropout=dropout, **kwargs)

    def forward(self, x: torch.Tensor, key_padding_mask: torch.BoolTensor = None) -> torch.Tensor:
        x = x.transpose(0, 1)
        attn = self.attention(query=x, key=x, value=x, key_padding_mask=key_padding_mask)[0]
        attn = attn.transpose(0, 1)
        return attn

In [80]:
from torchcrf import CRF

In [81]:
class F1Score:
    """
    Class for f1 calculation in Pytorch.
    """

    def __init__(self, average: str = 'weighted'):
        """
        Init.

        Args:
            average: averaging method
        """
        self.average = average
        if average not in [None, 'micro', 'macro', 'weighted']:
            raise ValueError('Wrong value of average parameter')

    @staticmethod
    def calc_f1_micro(predictions: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
        """
        Calculate f1 micro.

        Args:
            predictions: tensor with predictions
            labels: tensor with original labels

        Returns:
            f1 score
        """
        true_positive = torch.eq(labels, predictions).sum().float()
        f1_score = torch.div(true_positive, len(labels))
        return f1_score

    @staticmethod
    def calc_f1_count_for_label(
        predictions: torch.Tensor, labels: torch.Tensor, label_id: int
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Calculate f1 and true count for the label

        Args:
            predictions: tensor with predictions
            labels: tensor with original labels
            label_id: id of current label

        Returns:
            f1 score and true count for label
        """
        # label count
        true_count = torch.eq(labels, label_id).sum()

        # true positives: labels equal to prediction and to label_id
        true_positive = torch.logical_and(torch.eq(labels, predictions), torch.eq(labels, label_id)).sum().float()
        # precision for label
        precision = torch.div(true_positive, torch.eq(predictions, label_id).sum().float())
        # replace nan values with 0
        precision = torch.where(torch.isnan(precision), torch.zeros_like(precision).type_as(true_positive), precision)

        # recall for label
        recall = torch.div(true_positive, true_count)
        # f1
        f1 = 2 * precision * recall / (precision + recall)
        # replace nan values with 0
        f1 = torch.where(torch.isnan(f1), torch.zeros_like(f1).type_as(true_positive), f1)
        return f1, true_count

    def __call__(self, predictions: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
        """
        Calculate f1 score based on averaging method defined in init.

        Args:
            predictions: tensor with predictions
            labels: tensor with original labels

        Returns:
            f1 score
        """

        # simpler calculation for micro
        if self.average == 'micro':
            return self.calc_f1_micro(predictions, labels)

        f1_score = torch.tensor(0.0).type_as(predictions).float()
        for label_id in labels.unique():
            f1, true_count = self.calc_f1_count_for_label(predictions, labels, label_id)

            if self.average == 'weighted':
                f1_score += f1 * true_count
            elif self.average == 'macro':
                f1_score += f1

        if self.average == 'weighted':
            f1_score = torch.div(f1_score, len(labels))
        elif self.average == 'macro':
            f1_score = torch.div(f1_score, len(labels.unique()))

        return f1_score

In [82]:
class LayerNorm(nn.Module):
    """
    Layer Normalization
    paper: https://arxiv.org/abs/1607.06450
    """

    def __init__(self, normalized_shape: int):
        super(LayerNorm, self).__init__()
        self.layer_norm = nn.LayerNorm(normalized_shape)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.layer_norm(x)

In [83]:
class BiLSTMCRFAtt(nn.Module):
    """
    New model without nn.Embedding layer
    """

    def __init__(self, tag_to_idx: Dict, embeddings_dim: int = 100, hidden_dim: int = 4, spatial_dropout: float = 0.2):
        super().__init__()
        self.embedding_dim = embeddings_dim
        self.hidden_dim = hidden_dim
        self.tag_to_idx = tag_to_idx
        self.tagset_size = len(tag_to_idx.values())
        self.crf = CRF(self.tagset_size, batch_first=True)
        self.embedding_dropout = SpatialDropout(spatial_dropout)

        self.lstm = nn.LSTM(
            embeddings_dim, hidden_dim // 2, num_layers=2, bidirectional=True, batch_first=True, dropout=0.25
        )
        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, hidden_dim // 2)
        self.hidden2tag2 = nn.Linear(hidden_dim // 2, self.tagset_size)
        self.rnn_layer_norm = LayerNorm(hidden_dim)
        self.att = MultiHeadSelfAttention(embed_dim=hidden_dim, num_heads=2, dropout=0.25)

    def _get_lstm_features(self, embeds: torch.Tensor, lens: torch.Tensor, mask: bool) -> torch.Tensor:
        """
        LSTM forward

        Args:
            embeds: batch with embeddings
            lens: lengths of sequences
        """
        embeds = self.embedding_dropout(embeds)
        packed_embeds = torch.nn.utils.rnn.pack_padded_sequence(
            embeds, lens.cpu(), batch_first=True, enforce_sorted=False
        )
        lstm_out, self.hidden = self.lstm(packed_embeds)
        lstm_out, _ = torch.nn.utils.rnn.pad_packed_sequence(lstm_out, batch_first=True)
        lstm_out = self.rnn_layer_norm(lstm_out)
        lstm_out = self.att(lstm_out, key_padding_mask=mask)
        lstm_feats = self.hidden2tag2(self.hidden2tag(lstm_out.reshape(embeds.shape[0], -1, self.hidden_dim)))
        return lstm_feats

    def forward(
        self, embeds: torch.Tensor, lens: torch.Tensor, tags: torch.Tensor = None
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Forward

        Args:
            embeds: batch with embeddings
            lens: lengths of sequences
            tags: list of tags (optional)
        """
        mask1 = tags == self.tag_to_idx['PAD']
        lstm_feats = self._get_lstm_features(embeds, lens, mask1)

        if tags is not None:
            mask = tags != self.tag_to_idx['PAD']
            loss: torch.Tensor = self.crf(lstm_feats, tags, mask=mask)
            tag_seq = self.crf.decode(emissions=lstm_feats, mask=torch.tensor(mask))  # type: ignore

        else:
            loss = torch.tensor(0)
            tag_seq = self.crf.decode(lstm_feats)

        pred: torch.Tensor = torch.tensor([i for j in tag_seq for i in j]).type_as(embeds)
        return pred, -loss

In [84]:
model = BiLSTMCRFAtt(tag_to_idx, 100, 32, 0.2)
model

BiLSTMCRFAtt(
  (crf): CRF(num_tags=34)
  (embedding_dropout): SpatialDropout(
    (spatial_dropout): Dropout2d(p=0.2, inplace=False)
  )
  (lstm): LSTM(100, 16, num_layers=2, batch_first=True, dropout=0.25, bidirectional=True)
  (hidden2tag): Linear(in_features=32, out_features=16, bias=True)
  (hidden2tag2): Linear(in_features=16, out_features=34, bias=True)
  (rnn_layer_norm): LayerNorm(
    (layer_norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
  )
  (att): MultiHeadSelfAttention(
    (attention): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
    )
  )
)

In [85]:
metric = F1Score(average='weighted')
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [86]:
model.cuda();

In [87]:
embedding.cuda();

In [88]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
train_f1 = []
valid_f1 = []
for epoch in range(20):
    model.train()
    epoch_train_f1 = []
    for i, batch in enumerate(train_loader):
        tokens, lens, labels = batch
        tokens, lens, labels = tokens.cuda(), lens.cuda(), labels.cuda()
        optimizer.zero_grad()
        
        tag_seq, loss = model(embedding(tokens), lens, labels)
        
        loss.backward()
        optimizer.step()
        
        labels = labels.flatten()
        labels = labels[labels != tag_to_idx['PAD']]
        f1_score = metric(tag_seq, labels).item()
        epoch_train_f1.append(f1_score)
        
    model.eval()
    epoch_valid_f1 = []
    with torch.no_grad():
        for i, batch in enumerate(valid_loader):
            tokens, lens, labels = batch
            tokens, lens, labels = tokens.cuda(), lens.cuda(), labels.cuda()

            tag_seq, loss = model(embedding(tokens), lens, labels)
            labels = labels.flatten()
            labels = labels[labels != tag_to_idx['PAD']]
            f1_score = metric(tag_seq, labels).item()
            epoch_valid_f1.append(f1_score)
    
    mean_epoch_train_f1 = np.mean(epoch_train_f1)
    mean_epoch_valid_f1 = np.mean(epoch_valid_f1)
    train_f1.append(mean_epoch_train_f1)
    valid_f1.append(mean_epoch_valid_f1)
    print(f'{epoch=}. {mean_epoch_train_f1=:0.4f}. {mean_epoch_valid_f1=:0.4f}.')

<ipython-input-83-578e38282227>:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tag_seq = self.crf.decode(emissions=lstm_feats, mask=torch.tensor(mask))  # type: ignore


epoch=0. mean_epoch_train_f1=0.4651. mean_epoch_valid_f1=0.4471.
epoch=1. mean_epoch_train_f1=0.4825. mean_epoch_valid_f1=0.5258.
epoch=2. mean_epoch_train_f1=0.5771. mean_epoch_valid_f1=0.6321.
epoch=3. mean_epoch_train_f1=0.6593. mean_epoch_valid_f1=0.7369.
epoch=4. mean_epoch_train_f1=0.7162. mean_epoch_valid_f1=0.7902.
epoch=5. mean_epoch_train_f1=0.7626. mean_epoch_valid_f1=0.8190.
epoch=6. mean_epoch_train_f1=0.7847. mean_epoch_valid_f1=0.8364.
epoch=7. mean_epoch_train_f1=0.8071. mean_epoch_valid_f1=0.8418.
epoch=8. mean_epoch_train_f1=0.8229. mean_epoch_valid_f1=0.8581.
epoch=9. mean_epoch_train_f1=0.8294. mean_epoch_valid_f1=0.8574.
epoch=10. mean_epoch_train_f1=0.8355. mean_epoch_valid_f1=0.8674.
epoch=11. mean_epoch_train_f1=0.8444. mean_epoch_valid_f1=0.8719.
epoch=12. mean_epoch_train_f1=0.8490. mean_epoch_valid_f1=0.8778.
epoch=13. mean_epoch_train_f1=0.8485. mean_epoch_valid_f1=0.8764.
epoch=14. mean_epoch_train_f1=0.8558. mean_epoch_valid_f1=0.8845.
epoch=15. mean_epoch

In [89]:
idx_to_word = {v: k for k, v in word_to_idx.items()}
idx_to_tag = {v: k for k, v in tag_to_idx.items()}

In [90]:
labels = labels.cpu().detach().numpy()
tokens = tokens.cpu().detach().numpy()
tag_seq = tag_seq.cpu().detach().numpy()

In [91]:
correct_labels = []
correct_tag_seq = []
correct_tokens = []
for token in tokens:
    token = token[token != tag_to_idx['PAD']]
    correct_tokens.append(token)
    correct_labels.append(labels[:len(token)])
    labels = labels[len(token):]
    correct_tag_seq.append(tag_seq[:len(token)])
    tag_seq = tag_seq[len(token):]

In [92]:
for token, label, pred in zip(correct_tokens, correct_labels, correct_tag_seq):
    if not all(label == pred):
        label = label[token > 0]
        pred = pred[token > 0]
        token = token[token > 0]
        for t_, l_, p_ in zip([idx_to_word[t] for t in token],
                              [idx_to_tag[l] for l in label],
                              [idx_to_tag[p] for p in pred]):
            print(f'{t_}\t{l_}\t{p_}')
        print('-' * 50)

whos	O	O
got	O	O
the	O	O
most	O	O
popular	U-Rating	B-Dish
thin	B-Dish	I-Dish
crust	I-Dish	I-Dish
pizza	L-Dish	L-Dish
in	B-Location	B-Location
town	L-Location	L-Location
--------------------------------------------------
why	O	O
dont	O	O
we	O	O
go	O	O
to	O	O
a	O	O
nice	U-Rating	U-Rating
steak	U-Cuisine	B-Cuisine
house	O	L-Cuisine
--------------------------------------------------
will	O	O
i	O	O
be	O	O
close	U-Location	O
to	O	O
a	O	O
sea	B-Cuisine	B-Cuisine
food	L-Cuisine	L-Cuisine
restaurant	O	O
in	O	O
the	O	O
next	O	B-Location
20	B-Location	I-Location
minutes	L-Location	L-Location
--------------------------------------------------
will	O	O
the	O	O
japanese	U-Cuisine	U-Cuisine
restaurant	O	O
allow	B-Amenity	O
me	I-Amenity	O
to	I-Amenity	O
sing	I-Amenity	O
loudly	L-Amenity	O
while	O	O
i	O	O
eat	O	O
my	O	O
stir	B-Dish	B-Dish
fry	I-Dish	I-Dish
vegetables	L-Dish	L-Dish
--------------------------------------------------
will	O	O
the	O	O
mexican	U-Cuisine	U-Cuisine
restaurant	O	O
kick	O	B-Ame

In [93]:
print(f'{pd.__version__=}')
print(f'{spacy.__version__=}')
print(f'{eli5.__version__=}')
print(f'{sklearn.__version__=}')
print(f'{np.__version__=}')
print(f'{torch.__version__=}')

pd.__version__='1.2.4'
spacy.__version__='3.0.6'
eli5.__version__='0.11.0'
sklearn.__version__='0.23.2'
np.__version__='1.20.3'
torch.__version__='1.9.0'
